In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy.linalg import cho_factor, cho_solve
import os, glob
from astropy import units as u


import dill as pickle

import scipy.linalg as sl
import scipy.integrate as sint
import scipy.interpolate as sinter

import la_forge
import scipy.stats as sstats


from tqdm import tqdm
import json
import pandas as pd
import pyarrow.feather as feather

plt.style.use('dark_background')
mpl.rcParams['figure.figsize'] = [20.0, 7.0]


# Import Data

In [2]:
# Functions to read/write a feather file and return a python dictionary (portable)
def convert_value(value):
    """
    Convert NumPy arrays to native objects in a way that preserves shape.
    Scalars are converted to native Python types.
    Arrays with ndim > 1 are stored as a dictionary with 'data' and 'shape'.
    """
    if isinstance(value, np.ndarray):
        if value.ndim > 1:
            return {"data": value.flatten().tolist(), "shape": value.shape}
        else:
            return value.tolist()
    elif isinstance(value, np.generic):
        # Convert NumPy scalar to Python scalar.
        return value.item()
    else:
        return value

def write_fourier_dataframe(pta_model, filename):
    # Create a list of dictionaries – each row corresponds to one pulsar.
    rows = []
    for pulsar, params in pta_model.items():
        row = {"pulsar": pulsar}
        for key, value in params.items():
            row[key] = convert_value(value)
        rows.append(row)

    # Create a DataFrame from the rows.
    df = pd.DataFrame(rows)

    # Write the DataFrame to a Feather file.
    feather.write_feather(df, filename)

def restore_value(x):
    """
    Restore a value that was saved by convert_value().
    If x is a dict with keys 'data' and 'shape', rebuild a NumPy array with that shape.
    If x is a list, convert it to a NumPy array.
    Otherwise, return x unchanged.
    """
    if isinstance(x, dict) and "data" in x and "shape" in x:
        arr = np.array(x["data"])
        try:
            return arr.reshape(x["shape"])
        except Exception as e:
            # In case the reshape fails, return the flat array.
            return arr
    elif isinstance(x, list):
        return np.array(x)
    else:
        return x

def read_fourier_dataframe(filename):
    #df_loaded = feather.read_feather(f'{project_dir}/data/{dataset}-fourier-model.feather')
    df_loaded = feather.read_feather(filename)

    # First convert the DataFrame to a list of row dictionaries.
    records = df_loaded.to_dict(orient="records")

    # Reconstruct the original nested dictionary.
    pta_model = {}
    for row in records:
        pulsar = row["pulsar"]
        params = {}
        for key, value in row.items():
            if key != "pulsar":
                params[key] = restore_value(value)
        pta_model[pulsar] = params

    return pta_model

In [3]:

project_dir = './'
pta_model = read_fourier_dataframe(f'{project_dir}/pta_model_gpta_2.feather')

## par_dict 
This is for importing the parameters into dictionary

In [4]:
from pathlib import Path
import re
import json
import numpy as np
from typing import List, Dict

def load_param_names(path: str) -> List[str]:
   
    text = Path(path).read_text(encoding="utf-8")
    names = []
    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        parts = [p for p in re.split(r"[,\s]+", line) if p]
        names.extend(parts)
    if not names:
        raise ValueError("param_list.txt has no parameter names。")
    return names

def load_chain_matrix(path: str) -> np.ndarray:
   
    try:
        arr = np.loadtxt(path, dtype=float, comments="#", ndmin=2)
    except Exception:
        
        arr = np.genfromtxt(path, dtype=float, comments="#")
        arr = np.atleast_2d(arr)
    if arr.ndim != 2:
        raise ValueError("chain_1.txt has no two dimension matrix。")
    if arr.size == 0:
        raise ValueError("chain_1.txt is empty。")
    return arr

def build_sample_dicts(
    param_names: List[str],
    chain: np.ndarray,
    align: str = "left",
) -> List[Dict[str, float]]:
   
    n = len(param_names)
    m, k = chain.shape

    if k < n:
        raise ValueError(
            f"number of chain({k})is less than number of paramters ({n}),can't match , please check the file。"
        )
    if k > n:
        if align == "left":
            chain = chain[:, :n]
        elif align == "right":
            chain = chain[:, -n:]
        else:
            raise ValueError("align could either be 'left' or 'right'。")

    samples = [dict(zip(param_names, map(float, row))) for row in chain]
    return samples

if __name__ == "__main__":
    param_names = load_param_names("param_list.txt")
    chain = load_chain_matrix("chain_1.txt")
    big_array = build_sample_dicts(param_names, chain, align="left")
    with open("samples.json", "w", encoding="utf-8") as f:
        json.dump(big_array, f, ensure_ascii=False, indent=2)
    print(f" {len(big_array)} sampling, first few examples：")
    print(big_array[0])

 100001 sampling, first few examples：
{'J0030+0451_red_noise_gamma': 3.6039781182100143, 'J0030+0451_red_noise_log10_A': -10.043665246571784, 'J0030+0451_red_noise_log10_kappa': -17.569474403875617, 'J0034-0534_red_noise_gamma': 6.667684660836571, 'J0034-0534_red_noise_log10_A': -11.253477665420522, 'J0034-0534_red_noise_log10_kappa': -15.41479703965657, 'J0101-6422_red_noise_gamma': 6.72916524275286, 'J0101-6422_red_noise_log10_A': -15.273608194134036, 'J0101-6422_red_noise_log10_kappa': -16.909094101408694, 'J0102+4839_red_noise_gamma': 5.891793597698309, 'J0102+4839_red_noise_log10_A': -18.942072318865126, 'J0102+4839_red_noise_log10_kappa': -12.190698211238686, 'J0312-0921_red_noise_gamma': 6.401246316856674, 'J0312-0921_red_noise_log10_A': -10.225835203753324, 'J0312-0921_red_noise_log10_kappa': -16.70056008400239, 'J0340+4130_red_noise_gamma': 4.3356480961313935, 'J0340+4130_red_noise_log10_A': -12.241783490273706, 'J0340+4130_red_noise_log10_kappa': -10.393915542411081, 'J0418+6